# Custom_DataLoader

In [2]:
import torch
import torch.utils.data as data
import torchvision.datasets as dsets
import torchvision.transforms as transforms

In [4]:
# 하이퍼파라미터 설정
batch_size = 2

In [6]:
# 이미지폴더 & 데이터로더
img_dir = './images'
img_data = dsets.ImageFolder(img_dir, transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
]))
img_batch = data.DataLoader(img_data, batch_size=batch_size, shuffle=True, drop_last=True)

FileNotFoundError: Found no valid file for the classes .ipynb_checkpoints. Supported extensions are: .jpg, .jpeg, .png, .ppm, .bmp, .pgm, .tif, .tiff, .webp

# VGGNET

In [13]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.utils.data as data
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [16]:
# 하이퍼파라미터 설정
batch_size = 1
learning_rate = 0.0002
epoch = 100

In [11]:
# 데이터 로더
img_dir = './images'
img_data = dsets.ImageFolder(img_dir, transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
]))
img_batch = data.DataLoader(img_data, batch_size=batch_size, shuffle=True, num_workers=2)

FileNotFoundError: Found no valid file for the classes .ipynb_checkpoints. Supported extensions are: .jpg, .jpeg, .png, .ppm, .bmp, .pgm, .tif, .tiff, .webp

In [12]:
# 모델 정의
# Basic Blocks
def conv_2_block(in_dim, out_dim):
  model = nn.Sequential(
      nn.Conv2d(in_dim, out_dim, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.Conv2d(out_dim, out_dim, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.MaxPool2d(2, 2)
  )
  return model

def conv_3_block(in_dim, out_dim):
  model = nn.Sequential(
      nn.Conv2d(in_dim, out_dim, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.Conv2d(out_dim, out_dim, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.Conv2d(out_dim, out_dim, kernel_size=3, padding=1),
      nn.ReLU(),
      nn.MaxPool2d(2, 2)
  )
  return model

In [14]:
# VGG Model
class VGG(nn.Module):
  def __init__(self, base_dim, num_classes=2):
    super(VGG, self).__init__()
    self.feature = nn.Sequential(
        conv_2_block(3, base_dim),
        conv_2_block(base_dim, 2 * base_dim),
        conv_3_block(2 * base_dim, 4 * base_dim),
        conv_3_block(4 * base_dim, 8 * base_dim),
        conv_3_block(8 * base_dim, 8 * base_dim),
    )
    self.fc_layer = nn.Sequential(
        nn.Linear(8 * base_dim * 7 * 7, 100),
        nn.ReLU(),
        nn.Dropout(),
        nn.Linear(100, 20),
        nn.ReLU(),
        nn.Dropout(),
        nn.Linear(20, num_classes),
    )

  def forward(self, x):
    x = self.feature(x)
    x = x.view(x.size(0), -1)
    x = self.fc_layer(x)
    return x

model = VGG(base_dim=64).to(device)

for i in model.named_children():
  print(i)

('feature', Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (1): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (2): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (3): Seq

In [17]:
# optimizer & loss
loss_func = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 학습
for i in range(epoch):
  for X, Y in img_batch:
    X = X.to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    output = model(X)
    loss = loss_func(output, Y)
    loss.backward()
    optimizer.step()

  if i % 10 == 0:
    print(loss)

# GoogleNet

In [7]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.utils.data as data
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
# 하이퍼파라미터
batch_size = 1
learning_rate = 0.0002
epoch = 100

In [20]:
# 데이터로더
img_dir = './images'
img_data = dsets.ImageFolder(img_dir, transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
]))
img_batch = data.DataLoader(img_data, batch_size=batch_size, shuffle=True, num_workers=2)

FileNotFoundError: Found no valid file for the classes .ipynb_checkpoints. Supported extensions are: .jpg, .jpeg, .png, .ppm, .bmp, .pgm, .tif, .tiff, .webp

In [4]:
# 모델
# Base module
def conv_1(in_dim, out_dim):
  model = nn.Sequential(
      nn.Conv2d(in_dim, out_dim, 1, 1),
      nn.ReLU(),
  )
  return model

def conv_1_3(in_dim, mid_dim, out_dim):
  model = nn.Sequential(
      nn.Conv2d(in_dim, mid_dim, 1, 1),
      nn.ReLU(),
      nn.Conv2d(mid_dim, out_dim, 3, 1, 1),
      nn.ReLU(),
  )
  return model

def conv_1_5(in_dim, mid_dim, out_dim):
  model = nn.Sequential(
      nn.Conv2d(in_dim, mid_dim, 1, 1),
      nn.ReLU(),
      nn.Conv2d(mid_dim, out_dim, 5, 1, 2),
      nn.ReLU(),
  )
  return model

def max_3_1(in_dim, out_dim):
  model = nn.Sequential(
      nn.MaxPool2d(3, 1, 1),
      nn.Conv2d(in_dim, out_dim, 1, 1),
      nn.ReLU(),
  )
  return model

In [5]:
# Inception Module
class inception_module(nn.Module):
  def __init__(self, in_dim, out_dim_1, mid_dim_3, out_dim_3, mid_dim_5, out_dim_5, pool):
    super(inception_module, self).__init__()

    self.conv_1 = conv_1(in_dim, out_dim_1)
    self.conv_1_3 = conv_1_3(in_dim, mid_dim_3, out_dim_3)
    self.conv_1_5 = conv_1_5(in_dim, mid_dim_5, out_dim_5)
    self.max_3_1 = max_3_1(in_dim, pool)

  def forward(self, x):
    out_1 = self.conv_1(x)
    out_2 = self.conv_1_3(x)
    out_3 = self.conv_1_5(x)
    out_4 = self.max_3_1(x)
    output = torch.cat([out_1, out_2, out_3, out_4], 1)

    return output

In [8]:
# GoogLeNet
class GoogLeNet(nn.Module):
  def __init__(self, base_dim, num_classes=2):
    super(GoogLeNet, self).__init__()
    self.layer_1 = nn.Sequential(
        nn.Conv2d(3, base_dim, 7, 2, 3),
        nn.MaxPool2d(3, 2, 1),
        nn.Conv2d(base_dim, base_dim*3, 3, 1, 1),
        nn.MaxPool2d(3, 2, 1),
    )
    self.layer_2 = nn.Sequential(
        inception_module(base_dim * 3, 64, 96, 128, 16, 32, 32),
        inception_module(base_dim * 4, 128, 128, 192, 32, 96, 64),
        nn.MaxPool2d(3, 2, 1),
    )
    self.layer_3 = nn.Sequential(
        inception_module(480, 192, 96, 208, 16, 48, 64),
        inception_module(512, 160, 112, 224, 24, 64, 64),
        inception_module(512, 128, 128, 256, 24, 64, 64),
        inception_module(512, 112, 144, 288, 32, 64, 64),
        inception_module(528, 256, 160, 320, 32, 128, 128),
        nn.MaxPool2d(3, 2, 1),
    )
    self.layer_4 = nn.Sequential(
        inception_module(832, 256, 160, 320, 32, 128, 128),
        inception_module(832, 384, 192, 384, 48, 128, 128),
        nn.AvgPool2d(7, 1),
    )
    self.layer_5 = nn.Dropout2d(0.4)
    self.fc_layer = nn.Linear(1024, 1000)

  def forward(self, x):
    out = self.layer_1(x)
    out = self.layer_2(out)
    out = self.layer_3(out)
    out = self.layer_4(out)
    out = self.layer_5(out)
    out = out.view(out.size(0), -1)
    out = self.fc_layer(out)

    return out

model = GoogLeNet(base_dim=64).to(device)

for i in model.named_children():
  print(i)

('layer_1', Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (2): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
))
('layer_2', Sequential(
  (0): inception_module(
    (conv_1): Sequential(
      (0): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1))
      (1): ReLU()
    )
    (conv_1_3): Sequential(
      (0): Conv2d(192, 96, kernel_size=(1, 1), stride=(1, 1))
      (1): ReLU()
      (2): Conv2d(96, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU()
    )
    (conv_1_5): Sequential(
      (0): Conv2d(192, 16, kernel_size=(1, 1), stride=(1, 1))
      (1): ReLU()
      (2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (3): ReLU()
    )
    (max_3_1): Sequential(
      (0): MaxPool2d(kernel_size=3, stride=1, padding=1,

In [9]:
# optimizer & loss
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 학습
for i in range(epoch):
  for X, Y in img_batch:
    X = X.to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    output = model(X)

    loss = loss_func(output, Y)
    loss.backward()
    optimizer.step()

  if i % 10 == 0:
    print(loss)

# ResNet

In [10]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.utils.data as data
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [11]:
# 하이퍼 파라미터 설정
batch_size = 1
learning_rate = 0.0002
epoch = 100

In [12]:
# 데이터로더
img_dir = './images'
img_data = dsets.ImageFolder(img_dir, transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
]))

img_batch = data.DataLoader(img_data, batch_size=batch_size, shuffle=True, num_workers=2)

FileNotFoundError: [Errno 2] No such file or directory: './images'

In [13]:
# 모델 정의
# Basic Block
def conv_block_1(in_dim, out_dim, act_fn):
  model = nn.Sequential(
      nn.Conv2d(in_dim, out_dim, kernel_size=1, stride=1),
      act_fn,
  )
  return model

def conv_block_1_stride_2(in_dim, out_dim, act_fn):
  model = nn.Sequential(
      nn.Conv2d(in_dim, out_dim, kernel_size=1, stride=2),
      act_fn,
  )
  return model

def conv_block_1_n(in_dim, out_dim):
  model = nn.Sequential(
      nn.Conv2d(in_dim, out_dim, kernel_size=1, stride=1),
  )
  return model

def conv_block_1_stride_2_n(in_dim, out_dim):
  model = nn.Sequential(
      nn.Conv2d(in_dim, out_dim, kernel_size=1, stride=2),
  )
  return model

def conv_block_3(in_dim, out_dim, act_fn):
  model = nn.Sequential(
      nn.Conv2d(in_dim, out_dim, kernel_size=3, stride=1, padding=1),
      act_fn,
  )
  return model

In [14]:
# Bottle Neck Module
class BottleNeck(nn.Module):
  def __init__(self, in_dim, mid_dim, out_dim, act_fn):
    super(BottleNeck, self).__init__()
    self.layer = nn.Sequential(
        conv_block_1(in_dim, mid_dim, act_fn),
        conv_block_3(mid_dim, mid_dim, act_fn),
        conv_block_1_n(mid_dim, out_dim),
    )
    self.downsample = nn.Conv2d(in_dim, out_dim, 1, 1)

  def forward(self, x):
    downsample = self.downsample(x)
    out = self.layer(x)
    out = out + downsample

    return out

class BottleNeck_no_down(nn.Module):
  def __init__(self, in_dim, mid_dim, out_dim, act_fn):
    super(BottleNeck_no_down, self).__init__()
    self.layer = nn.Sequential(
        conv_block_1(in_dim, mid_dim, act_fn),
        conv_block_3(mid_dim, mid_dim, act_fn),
        conv_block_1_n(mid_dim, out_dim),
    )
  def forward(self, x):
    out = self.layer(x)
    out = out + x

    return out

class BottleNeck_stride(nn.Module):
  def __init__(self, in_dim, mid_dim, out_dim, act_fn):
    super(BottleNeck_stride, self).__init__()
    self.layer = nn.Sequential(
        conv_block_1_stride_2(in_dim, mid_dim, act_fn),
        conv_block_3(mid_dim, mid_dim, act_fn),
        conv_block_1_n(mid_dim, out_dim),
    )
    self.downsample = nn.Conv2d(in_dim, out_dim, 1, 2)

  def forward(self, x):
    downsample = self.downsample(x)
    out = self.layer(x)
    out = out + downsample

    return out

In [16]:
# ResNet Model
class ResNet(nn.Module):
  def __init__(self, base_dim, num_classes=2):
    super(ResNet, self).__init__()
    self.act_fn = nn.ReLU()
    self.layer_1 = nn.Sequential(
        nn.Conv2d(3, base_dim, 7, 2, 3),
        nn.ReLU(),
        nn.MaxPool2d(3, 2, 1),
    )
    self.layer_2 = nn.Sequential(
        BottleNeck(base_dim, base_dim, base_dim * 4, self.act_fn),
        BottleNeck_no_down(base_dim * 4, base_dim, base_dim * 4, self.act_fn),
        BottleNeck_stride(base_dim * 4, base_dim, base_dim * 4, self.act_fn),
    )
    self.layer_3 = nn.Sequential(
        BottleNeck(base_dim * 4, base_dim * 2, base_dim * 8, self.act_fn),
        BottleNeck_no_down(base_dim * 8, base_dim * 2, base_dim * 8, self.act_fn),
        BottleNeck_no_down(base_dim * 8, base_dim * 2, base_dim * 8, self.act_fn),
        BottleNeck_stride(base_dim * 8, base_dim * 2, base_dim * 8, self.act_fn),
    )
    self.layer_4 = nn.Sequential(
        BottleNeck(base_dim * 8, base_dim * 4, base_dim * 16, self.act_fn),
        BottleNeck_no_down(base_dim * 16, base_dim * 4, base_dim * 16, self.act_fn),
        BottleNeck_no_down(base_dim * 16, base_dim * 4, base_dim * 16, self.act_fn),
        BottleNeck_no_down(base_dim * 16, base_dim * 4, base_dim * 16, self.act_fn),
        BottleNeck_no_down(base_dim * 16, base_dim * 4, base_dim * 16, self.act_fn),
        BottleNeck_stride(base_dim * 16, base_dim * 4, base_dim * 16, self.act_fn),
    )
    self.layer_5 = nn.Sequential(
        BottleNeck(base_dim * 16, base_dim * 8, base_dim * 32, nn.ReLU()),
        BottleNeck_no_down(base_dim * 32, base_dim * 8, base_dim * 32, self.act_fn),
        BottleNeck(base_dim * 32, base_dim * 8, base_dim * 32, self.act_fn),
    )
    self.avgpool = nn.AvgPool2d(7, 1)
    self.fc_layer = nn.Linear(base_dim * 32, num_classes)

  def forward(self, x):
    out = self.layer_1(x)
    out = self.layer_2(out)
    out = self.layer_3(out)
    out = self.layer_4(out)
    out = self.layer_5(out)
    out = self.avgpool(out)
    out = out.view(out.size(0), -1)
    out = self.fc_layer(out)

    return out

model = ResNet(base_dim=64).to(device)

for i in model.named_children():
  print(i)

('act_fn', ReLU())
('layer_1', Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
))
('layer_2', Sequential(
  (0): BottleNeck(
    (layer): Sequential(
      (0): Sequential(
        (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
        (1): ReLU()
      )
      (1): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU()
      )
      (2): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
      )
    )
    (downsample): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
  )
  (1): BottleNeck_no_down(
    (layer): Sequential(
      (0): Sequential(
        (0): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1))
        (1): ReLU()
      )
      (1): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU()
      )
   

In [17]:
# optimizer & loss
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [18]:
# 학습
for i in range(epoch):
  for X, Y in img_batch:
    X = X.to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    output = model(X)
    loss = loss_func(output, Y)
    loss.backward()
    optimizer.step()

  if i % 10 == 0:
    print(i)

NameError: name 'img_batch' is not defined